In [13]:
from utils import load_data
import pandas as pd
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import batch, topn, util
from lenskit import crossfold as xf
%matplotlib inline

In [22]:
original = load_data("./active1000")
original.head()

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId
0,2029361762,None,NaN,None,http://adressa.no,cx:i9i5zdr4ns9bm4ky:2by1rj0hmjgy,None,1483225227,None
1,351617637,None,41.0,None,http://adressa.no,cx:iimz2wwcwxu7d721:2r8odp9zhg5yp,None,1483225320,None
2,483221791,None,NaN,None,http://adressa.no/nyheter/trondheim/2016/12/31...,cx:iimz2wwcwxu7d721:2r8odp9zhg5yp,None,1483225328,None
3,929295193,None,11.0,None,http://adressa.no,cx:idlec1eyfbpru62o:2yhui5zv6k12b,None,1483225334,None
4,1908339310,None,NaN,None,http://adressa.no/pluss/kultur/2016/12/31/bare...,cx:iimz2wwcwxu7d721:2r8odp9zhg5yp,None,1483225387,None


In [30]:
dataset = original.copy()
dataset = dataset[pd.notnull(dataset["documentId"])]
dataset = dataset.rename(columns={'userId': 'user', 'documentId': 'item'})
dataset.head()

,eventId,category,activeTime,title,url,user,publishtime,time,item
15,828764961,None,NaN,Se lesernes nyttårsbilder,http://adressa.no/nyheter/2016/12/31/se-lesern...,cx:il0sdznsjgg9uxgy:3bi2ksost85yi,2016-12-31T17:13:57.000Z,1483225500,9f3999bd1a1a8d67bcb073ad54840f15cb30f014
17,378480578,None,7.0,Se lesernes nyttårsbilder,http://adressa.no/nyheter/2016/12/31/se-lesern...,cx:ijtjdxpz93t5f8m5:2iz9n4nuh22ky,2016-12-31T17:13:57.000Z,1483225504,9f3999bd1a1a8d67bcb073ad54840f15cb30f014
18,1899017717,None,NaN,Måtte i oppvaskmøte etter å ha kritisert lands...,http://adressa.no/100sport/langrenn_old/matte-...,cx:ilfpxvtxtbw8zoiz:jy0kpshctu1c,None,1483225511,40d2ee3cfa2a7119b197e293353d0e2401782b1a
23,506807602,None,60.0,Se lesernes nyttårsbilder,http://adressa.no/nyheter/2016/12/31/se-lesern...,cx:ep1jmztcea6w2dtv4kax775yy:1sm7epvktvk6k,2016-12-31T17:13:57.000Z,1483225561,9f3999bd1a1a8d67bcb073ad54840f15cb30f014
24,1187690539,None,10.0,Måtte i oppvaskmøte etter å ha kritisert lands...,http://adressa.no/100sport/langrenn_old/matte-...,cx:13785776217372059747227:y69zdttav1l7,None,1483225575,40d2ee3cfa2a7119b197e293353d0e2401782b1a


In [31]:
algo_ii = knn.ItemItem(20, feedback='implicit')
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs


In [32]:
all_recs = []
test_data = []
for train, test in xf.partition_users(dataset[['user', 'item']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))

In [33]:
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs.head()

,item,score,user,rank,Algorithm
0,21124879767ab94be2415310b1e04c380051b59f,11.171695,cx:13082126012361420762846:3nhzg71lk2zqc,1,ItemItem
1,a4108a6448e235d895c62807f493d918a9b8125e,11.011790,cx:13082126012361420762846:3nhzg71lk2zqc,2,ItemItem
2,cf83d342459ce871e2a8562a91b7dca946e3201a,10.734710,cx:13082126012361420762846:3nhzg71lk2zqc,3,ItemItem
3,6af41040924f1aa5cbc5427af8ecb63df0b36fc3,10.706119,cx:13082126012361420762846:3nhzg71lk2zqc,4,ItemItem
4,494a1f550157cc78c7ef55dd2498faa450bdaa5b,10.688633,cx:13082126012361420762846:3nhzg71lk2zqc,5,ItemItem


In [34]:
test_data = pd.concat(test_data, ignore_index=True)

In [35]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
results.head()

results.groupby('Algorithm').ndcg.mean()

truth index not unique: may have duplicate items
Empty DataFrame
Columns: []
Index: [(0, 43b008750bfd8535d57f3788d603ce80bdfb6e43), (0, d698b58803566019642f15e9d19ed06f20fe38f0), (0, 76e2f91b98ffcbcbd7cbd92f66319d78ee6e4af1), (0, dff3471f505424723a63181f46383844011a976f), (0, e0719cedc76796a83a94c4646d2220eef2f324f1), (0, c47f63e7e6d046709c8222e5baebec3ba19e1ae8), (0, 1907d9891155cadbf240e6774cfb9246e1248561), (0, f59688e88ce057b388b573e7740bf7998496e46c), (0, 5bf4087aef39ff8d08c642d394335e50fe10cc1d), (0, 23aef9831d5bd956bf12fb388799d2339ef8f5d2), (0, e6a50f7520b45ccef4fb5e1c9f86c6336aa13786), (0, dad570826871d3a9678222ff6396e5297a148deb), (0, 042fe9b766831f187360d2d432571345a0d4b00b), (0, a893aab473b64b0580fd9b930ca3ffc6bbc2cd53), (0, f89d5df0e8cfd384b78fb2a02d6c96e70b63a37a), (0, 637dcd5bc42e2b0bba3370ae32703c2c0bffa4c7), (0, ccc2e4530968515ecc184bddd771495a6244b089), (0, f874f1912d057d06afbd45cb3e2f39042d14846b), (0, acd23789ab8dc56e8262a8bc36fb968f8dc88c1c), (0, 290fcf99f1d51b15ff

Algorithm
ItemItem    0.12895
Name: ndcg, dtype: float64